In [11]:
#Testing with the given example..

import numpy as np
claims=[('P1','Q1'), ('P1', 'Q2'), ('P2','' )]
embeddings={'P1': [1, 2, 3], 'P2': [4, 5, 6], 'Q1': [3, 2, 1], 'Q2': [1, 0, 1]}
count=0
value=[]
for x in claims:
    for y in x:
        
        if(y!=''):
            count=count+1
            value.append(list(embeddings[y]))
            
res = list()
doc_embedding=[]

for j in range(0, len(value[0])): 
    tmp = 0
    for i in range(0, len(value)): 
        tmp = tmp + value[i][j] 
    res.append(float(tmp))
    
for k in res:
    doc_embedding.append(k/count)
    
print(doc_embedding)


[2.0, 2.2, 2.8]


In [7]:
#Main Method 
# If there is no embedding for a given property (P#) or entity (Q#) then an embedding of 0s
# ([0.0, 0.0, 0.0] is used as part of the averaging.
import numpy as np
def claims_to_doc_embedding(claims, embeddings):
    count=0
    start=True
    embedding_Sum=[]
    for x in claims:
        for y in x:
            if(y!=''):
                count=count+1
                if y in embeddings :
                    if start or count==1:
                        embedding_Sum=np.array(embeddings[y])
                        start=False
                    else:
                        embedding_Sum=np.add(embedding_Sum,np.array(embeddings[y]))
       
    doc_embedding=np.divide(embedding_Sum,float(count))
    return list(doc_embedding)
 

In [8]:
    
doc_embedding=claims_to_doc_embedding(([('P1','Q1'), ('P1', 'Q2'), ('P2','' )]),({'P1': [1, 2, 3], 'P2': [4, 5, 6], 'Q1': [3, 2, 1], 'Q2': [1, 0, 1]}))
print(doc_embedding) 

[2.0, 2.2, 2.8]


In [13]:
#Not giving Q2 in embedding and checking if it adds [0.0,0.0,0.0] in the average instead of [1,0,1].
doc_embedding=claims_to_doc_embedding(([('P1','Q1'), ('P1', 'Q2'), ('P2','' )]),({'P1': [1, 2, 3], 'P2': [4, 5, 6], 'Q1': [3, 2, 1]}))
print(doc_embedding) 
#its working as expected!!

[1.8, 2.2, 2.6]


In [14]:
!pip install fasttext

  Using cached https://files.pythonhosted.org/packages/10/61/2e01f1397ec533756c1d893c22d9d5ed3fce3a6e4af1976e0d86bb13ea97/fasttext-0.9.1.tar.gz
  Running setup.py bdist_wheel for fasttext ... error
  Complete output from command /srv/paws/bin/python3.6 -u -c "import setuptools, tokenize;__file__='/tmp/pip-build-demwd2rn/fasttext/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d /tmp/tmp65swjc2ypip-wheel- --python-tag cp36:
  usage: -c [global_opts] cmd1 [cmd1_opts] [cmd2 [cmd2_opts] ...]
     or: -c --help [cmd1 cmd2 ...]
     or: -c --help-commands
     or: -c cmd --help
  
  error: invalid command 'bdist_wheel'
  
  ----------------------------------------
  Failed building wheel for fasttext
  Running setup.py clean for fasttext
Failed to build fasttext
  Running setup.py install for fasttext ... done


In [15]:
#Another way to find document embedding using fasttext module..Method 2

import numpy as np
import fasttext
model=fasttext.load_model('model.bin')

#Firstly, get the claims of a particular Qid using qid_to_claims function..
def qid_to_claims(qid):
    
    import requests

    S = requests.Session()

    URL = "http://www.wikidata.org/w/api.php"
    PARAMS = {
    "action": "wbgetentities",
    "ids": qid,
    "props": "claims",
    "format": "json"
         }

    R = S.get(url=URL, params=PARAMS)
    DATA = R.json()
    dict=DATA['entities'][qid]['claims'].keys()

    Claims=[]

    for i in dict:
         length= len(DATA['entities'][qid]['claims'][i])
    
         for j in range(0,length):
            if('datavalue' in DATA['entities'][qid]['claims'][i][j]['mainsnak'].keys() ):
                if(DATA['entities'][qid]['claims'][i][j]['mainsnak']['datavalue']['type']=='wikibase-entityid'):
                    Claims.append(tuple((i, DATA['entities'][qid]['claims'][i][j]['mainsnak']['datavalue']['value']['id'])))
            
                else:
                    Claims.append(tuple((i,"")))

    return Claims

claims=qid_to_claims('Q223') #As an example take qid="Q223"
embedding=[]
count=0
for i in range(len(claims)):
        pid=claims[i][0]
        qid=claims[i][1]
        
        if i==0:
            embedding=np.array(model.get_word_vector(pid)) #Embedding is of type-numpy array.
        else:
            embedding=np.add(embedding,np.array(model.get_word_vector(pid)))
        count=count+1
        if qid!='':
            embedding=np.add(embedding,np.array(model.get_word_vector(qid)))
            count=count+1
doc_embedding=np.divide(embedding,count)

#document embedding of "Q223"
print(doc_embedding)


[ 0.1743639  -0.7817916  -0.24616419 -0.01156415 -0.07600879 -0.0842642
  0.14212248  0.22452301 -0.31397432 -0.11713884 -0.28759632 -0.15209016
 -0.11054951 -0.12030383 -0.32426217 -0.45531902 -0.20288399  0.68282366
 -0.01025043 -0.39958563 -0.06441876 -0.3568137  -0.33642107  0.24331342
 -0.62871826 -0.37913838 -0.23013517 -0.32640928 -0.28306362 -0.04336843
 -0.4205433  -0.35524622  0.36258218 -0.03363707  0.39069888 -0.62807715
 -0.05582381  0.53624886  0.07011648  0.23984833 -0.1314159  -0.2877831
 -0.32883877 -0.18838678  0.16652696  0.28536546  0.4498171   0.02313693
  0.35487166  0.10588682]
